*Huomaa: käyttääksesi tätä työkirjaa, ympäristössäsi on oltava asennettuna paketit numpy ja matplotlib.*

In [ ]:
from math import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


# Kuvanmuodostus ja näytteistys

Tässä luvussa tutustumme kuviin, niiden esitysmuotoihin ja matemaattisiin
tulkintoihin. Opimme mitä signaalit ovat ja mitä näytteistys ja interpolointi
tarkoittavat. Tutustumme myös kuvien yksinkertaisiin matemaattisiin
operaatioihin ja kohteiden irrottamiseen kynnystyksen avulla.

Käytännöllisiä asioita tällä kerralla:

* näytteistyksen kokeilua yksiulotteisella signaalilla,
* kuvamatematiikkaa,
* kuvien kynnystystä.

## Kuvanmuodostus

Palautetaan ensimmäisenä mieleen, millä tavoin kuvat muodostuvat. Sama periaate
pätee niin alkeelliseen neulanreikäkameraan ja objektiivilla varustettuun
filmikameraan tai digitaaliseen kameraan kuin myös elävien olentojen silmiin.

Kuva muodostuu, kun tarkasteltava kolmiulotteinen näkymä projisoidaan eli
heijastetaan kaksiulotteiselle pinnalle. Tältä pinnalta kuva voidaan taltioida
valokuvausfilmille, lukea digitaalisella kuvasensorilla tai havaita silmän
verkkokalvon valoherkillä aistinsoluilla. Tällä tavoin *jatkuvasta*
kuvasignaalista muodostuu *diskreetti* eli eroteltu, epäjatkuva tallenne.
Tällaista epäjatkuvaa signaalia kutsutaan yleensä *näytteistetyksi* signaaliksi.

Huomautus: koska näkymästä heijastuva valo koostuu joukosta erillisiä fotoneja,
kuvasignaalia voitaisiin pitää tietyssä mielessä diskreettinä. On mahdollista
rakentaa laitteita, jotka mittaavat yksittäisiä fotoneja. On myös kehitetty
*valokenttäkameroita* (engl. *light field camera*), jotka mittaavat yksittäisten
fotonien saapumissuunnan. Koska tyypillisissä valokuvissa fotonien määrä on
kuitenkin sangen suuri, voidaan tämän luvun kannalta katsoa valon intensiteetti
jatkuvaksi funktioksi, joka muodostuu jonkinlaisesta todennäköisyysjakaumasta
fotonin havaitsemiselle tietyssä kuvapinnan pisteessä. Kvanttimekaniikan ilmiöt
ja epätarkkuusperiaate tekevät fotonien havaitsemisesta joka tapauksessa enemmän
tai vähemmän epätarkkaa.

## Signaalit


Signaalit kuvaavat erilaisia kiinnostavia fyysisiä suureita, jotka vaihtelevat
ajan suhteen ja joskus myös tilan suhteen. Tyypillisesti signaaleissa
kiinnostavaa ovat nimenomaan jonkin fyysisen suureen, kuten äänenpaineen tai
valon intensiteetin *vaihtelut*, joita voidaan kuvata eri taajuuksilla
tapahtuvien säännöllisten muutosten summana. Näitä niin kutsuttuja
*taajuuskomponentteja* voidaan kuvata siniaaltoina, joiden heilahtelunopeus,
heilahtelun laajuus ja heilahtelun alkamiskohta vaihtelevat. Myöhemmin, kun
käsittelemme Fourier-muunnosta, palaamme tähän aiheeseen tarkemmin. Tässä
vaiheessa riittää ymmärtää, että mikä tahansa signaali on purettavissa summaksi
tällaisia säännöllisiä vaihteluita.

### Tehtävä 2.1

Seuraavassa koodiesimerkissä rakennetaan keinotekoisia signaaleja summaamalla siniaaltoina esitettyjä taajuuskomponentteja. Voit kokeilla eri parametrien arvoja ja havainnoida niiden vaikutuksia. Kokeile muuttaa numeroarvoja kahdessa parametrilistassa (*amplitudes* ja *phases*). Halutessasi voit kokeilla muuttaa myös komponenttien määrää ja taajuuksia (*frequencies*), mutta pidä huoli siitä, että taulukoissa on oikea määrä elementtejä.

Solun koodin voi suorittaa painamalla Shift+Enter.

Kerro havainnoistasi ja yritä selvittää itsellesi, mitä kukin parametreista *frequency*, *amplitude*, *phase* tekee. Huomioi, miten kutakin niistä käytetään funktion arvon laskemiseen. Mitkä asiat vaikuttavat lopputuloksena syntyvään signaaliin, joka on piirretty kuvaan paksulla vihreällä viivalla?

In [ ]:
x = np.linspace(0, 2*pi, num=100, endpoint=True)
components = 5
y_offset = 8 # moves signal up so that the value is always non-negative; may need to be changed based on amplitudes
frequencies = [1, 2, 7, 4, 5]
amplitudes = [2, 0.5, 1, 2, 0.5]
phases = [0, pi/4, pi/8, 3*pi/4, 0]

def freq_comp(i):
    return lambda x : amplitudes[i] * np.sin(frequencies[i] * x + phases[i])

def signal(x):
    return sum([freq_comp(i)(x) for i in range(0, components)])

for i in range(0, components):
    _ = plt.plot(x, freq_comp(i)(x) + y_offset, 'r:')

_ = plt.plot(x, signal(x) + y_offset, 'g-', linewidth=2)

## Näytteistys

Signaalit ovat yleensä *jatkuvia*, mutta elektronisilla laitteilla voidaan
analysoida lähinnä yksittäisinä numeroina tallennettavia suureita.
**Näytteistys** tarkoittaa jatkuvan signaalin tallentamista lukemalla signaalin
arvoja rajallisessa joukossa näytepisteitä. Ajan suhteen muuttuvaa signaalia,
kuten ääntä, voidaan näytteistää lukemalla signaalin voimakkuutta tietyin
väliajoin. Kuvasignaali muuttuu sekä tilassa että ajassa, joten kuvasignaalia
näytteistetään lukemalla valon kirkkautta yhtä aikaa useista näytepisteistä
pinnalta, jolle kuva on projisoitu.

Niinsanottu *Nyquistin näytteistystaajuus* (engl. *Nyquist rate*) liittyy
signaalin muutostaajuuteen. Jos signaalin nopeimmat muutokset tapahtuvat
taajuudella $k$ kertaa sekunnissa, silloin näytteistämällä $2k$ näytettä
sekunnissa saadaan signaalin muutokset talteen riittävällä tarkkuudella sen
uudelleenrakentamista varten. Tällöin taajuus $2k$ on tämän signaalin Nyquistin
näytteistystaajuus.

Seuraavassa koodiesimerkissä näytteistetään edellä generoitua signaalia.
Muuttujan *signal_frequency* pitäisi olla sama kuin suurin arvo taulukossa
*frequencies* edellä. Tässä se on jätetty vapaavalintaiseksi, jotta eri
arvojen vaikutusta voi kokeilla.

In [ ]:
signal_frequency = 5
x_sampled = np.linspace(0, 2 * pi, 2 * signal_frequency, endpoint=True)
y_sampled = signal(x_sampled) + 8
p = plt.plot(x, signal(x) + 8, 'g-', linewidth=2)
p = plt.stem(x_sampled, y_sampled, 'b:')

In [ ]:
from scipy.interpolate import interp1d

sig_linear = interp1d(x_sampled, y_sampled)
sig_cubic = interp1d(x_sampled, y_sampled, kind='cubic')

p = plt.plot(x, signal(x) + 8, 'go')
p = plt.plot(x, sig_linear(x), 'b-')
p = plt.plot(x, sig_cubic(x), 'r-')
p = plt.plot(x_sampled, y_sampled, 'bo')